In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.5):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)  
        self.fc2 = nn.Linear(hidden_size, int(hidden_size / 2))
        self.dropout2 = nn.Dropout(dropout_rate)  
        self.fc3 = nn.Linear(int(hidden_size / 2), output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        # out = self.dropout1(out)  
        out = self.fc2(out)
        out = self.relu(out)
        # out = self.dropout2(out)  
        out = self.fc3(out)
        return out
    
input_size = 365*19
hidden_size = 1024
output_size = 1  
model = MLP(input_size, hidden_size, output_size)
model.to('cuda')
train_dataset = torch.load('../../../data/cleaned/train.pt')
train_x, train_y = train_dataset.tensors  

val_dataset = torch.load('../../../data/cleaned/val.pt')
test_x, test_y = val_dataset.tensors  

train_x = train_x.reshape(-1, 365*19)
test_x = test_x.reshape(-1, 365*19)

train_x = train_x.to('cuda')
test_x = test_x.to('cuda')
train_y = train_y.to('cuda')
test_y = test_y.to('cuda')

print(train_x.shape)
criterion = nn.MSELoss()  

optimizer = optim.Adam(model.parameters(), lr=5e-5)
num_epochs = 1500

for epoch in range(num_epochs):
    model.train()  
    outputs = model(train_x)
    loss = criterion(outputs, train_y.unsqueeze(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 100 == 0:
        model.eval()  
        with torch.no_grad():
            val_outputs = model(test_x)
            val_loss = criterion(val_outputs, test_y.unsqueeze(1))
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {np.sqrt(loss.item()):.4f}, Validation Loss: {np.sqrt(val_loss.item()):.4f}')

torch.Size([17724, 6935])
Epoch [100/1500], Loss: 287.5648, Validation Loss: 128.6548
Epoch [200/1500], Loss: 284.1812, Validation Loss: 128.4311
Epoch [300/1500], Loss: 277.7788, Validation Loss: 131.1986
Epoch [400/1500], Loss: 263.9069, Validation Loss: 130.8083
Epoch [500/1500], Loss: 233.2045, Validation Loss: 133.5549
Epoch [600/1500], Loss: 181.4070, Validation Loss: 134.0724
Epoch [700/1500], Loss: 129.1598, Validation Loss: 131.5045
Epoch [800/1500], Loss: 95.1695, Validation Loss: 124.3192
Epoch [900/1500], Loss: 75.0208, Validation Loss: 120.4086
Epoch [1000/1500], Loss: 63.8049, Validation Loss: 118.8566
Epoch [1100/1500], Loss: 57.0620, Validation Loss: 116.6111
Epoch [1200/1500], Loss: 52.3870, Validation Loss: 115.3359
Epoch [1300/1500], Loss: 48.6233, Validation Loss: 114.4863
Epoch [1400/1500], Loss: 45.2549, Validation Loss: 114.8049
Epoch [1500/1500], Loss: 42.0685, Validation Loss: 115.1563


In [1]:
# for creatinig losses over 100 epochs

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.5):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)  
        self.fc2 = nn.Linear(hidden_size, int(hidden_size / 2))
        self.dropout2 = nn.Dropout(dropout_rate)  
        self.fc3 = nn.Linear(int(hidden_size / 2), output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        # out = self.dropout1(out)  
        out = self.fc2(out)
        out = self.relu(out)
        # out = self.dropout2(out)  
        out = self.fc3(out)
        return out
    
input_size = 365*19
hidden_size = 1024
output_size = 1  
model = MLP(input_size, hidden_size, output_size)
model.to('cuda')
train_dataset = torch.load('../../../data/cleaned/train.pt')
train_x, train_y = train_dataset.tensors  

val_dataset = torch.load('../../../data/cleaned/val.pt')
test_x, test_y = val_dataset.tensors  

train_x = train_x.reshape(-1, 365*19)
test_x = test_x.reshape(-1, 365*19)

train_x = train_x.to('cuda')
test_x = test_x.to('cuda')
train_y = train_y.to('cuda')
test_y = test_y.to('cuda')

print(train_x.shape)
criterion = nn.MSELoss()  

optimizer = optim.Adam(model.parameters(), lr=5e-5)
num_epochs = 100
train_losses = []
val_losses = []
for epoch in range(num_epochs):
    model.train()  
    outputs = model(train_x)
    loss = criterion(outputs, train_y.unsqueeze(1))
    optimizer.zero_grad()
    loss.backward()
    train_losses.append(np.sqrt(loss.item()))
    optimizer.step()
    if (epoch + 1) % 1 == 0:
        model.eval()  
        with torch.no_grad():
            val_outputs = model(test_x)
            val_loss = criterion(val_outputs, test_y.unsqueeze(1))
            val_losses.append(np.sqrt(val_loss.item()))
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {np.sqrt(loss.item()):.4f}, Validation Loss: {np.sqrt(val_loss.item()):.4f}')



torch.Size([17724, 6935])
Epoch [1/100], Loss: 300.0679, Validation Loss: 134.2102
Epoch [2/100], Loss: 299.9646, Validation Loss: 134.0870
Epoch [3/100], Loss: 299.8718, Validation Loss: 133.9628
Epoch [4/100], Loss: 299.7782, Validation Loss: 133.8347
Epoch [5/100], Loss: 299.6811, Validation Loss: 133.6985
Epoch [6/100], Loss: 299.5776, Validation Loss: 133.5538
Epoch [7/100], Loss: 299.4672, Validation Loss: 133.4006
Epoch [8/100], Loss: 299.3503, Validation Loss: 133.2380
Epoch [9/100], Loss: 299.2259, Validation Loss: 133.0667
Epoch [10/100], Loss: 299.0945, Validation Loss: 132.8873
Epoch [11/100], Loss: 298.9564, Validation Loss: 132.6996
Epoch [12/100], Loss: 298.8113, Validation Loss: 132.5031
Epoch [13/100], Loss: 298.6587, Validation Loss: 132.2974
Epoch [14/100], Loss: 298.4982, Validation Loss: 132.0829
Epoch [15/100], Loss: 298.3300, Validation Loss: 131.8597
Epoch [16/100], Loss: 298.1538, Validation Loss: 131.6278
Epoch [17/100], Loss: 297.9697, Validation Loss: 131.38

In [2]:
print(train_losses)

[300.067882944843, 299.964633331998, 299.87175644348366, 299.7782383487834, 299.68110655328275, 299.5775932458901, 299.46724831356767, 299.35027298718137, 299.22587622062366, 299.094466682351, 298.95638791469236, 298.81126463873477, 298.65869420627956, 298.4982464353853, 298.3300002723494, 298.1537983994167, 297.96965292207193, 297.7777983455113, 297.57841666105423, 297.3718868058983, 297.1582727478742, 296.9374934224373, 296.7096908174723, 296.4749936756893, 296.2335834058995, 295.9857207282135, 295.73169322039195, 295.47197413629607, 295.2068527956287, 294.93675169432515, 294.66214763691653, 294.38365167753454, 294.10182388502795, 293.81737259307863, 293.5310087963451, 293.24353946421394, 292.9558147144378, 292.66866130916713, 292.3830829365817, 292.1000203269421, 291.8205382808071, 291.54566537679824, 291.27648752946055, 291.0141077980585, 290.75956562080637, 290.5139438348872, 290.2782743549713, 290.0535241554565, 289.840635091079, 289.64040205054266, 289.453580043502, 289.28065416

In [3]:
print(val_losses)

[134.21015324622425, 134.08696600760643, 133.96279071770266, 133.83474972657513, 133.6985444245748, 133.55381347513068, 133.40058037261306, 133.2380022154903, 133.06673460852267, 132.88733920834972, 132.69959171235232, 132.5030807424869, 132.29738340666455, 132.08291518729817, 131.85970734790442, 131.6278341337804, 131.38790374307675, 131.14040811521443, 130.8860877585926, 130.62526166241736, 130.35828032771835, 130.0856839019959, 129.80801629233073, 129.5259988380711, 129.24043424466663, 128.95219257054143, 128.66240952585957, 128.37213969549623, 128.0824929488804, 127.79478407171985, 127.51043921724605, 127.23097767323608, 126.95803163595046, 126.69334285983616, 126.43867785536393, 126.19600098284613, 125.96725765054981, 125.75448472991927, 125.55968451298371, 125.3849151737361, 125.23223348917602, 125.10373039751852, 125.00121093163457, 124.92644710988942, 124.88105278123659, 124.86657723095881, 124.88385228883676, 124.93383405076865, 125.01734254694426, 125.13376436587768, 125.2828